Per ciascuna squadra del campionato di Serie A viene determinata la formazione teoricamente più efficace, confrontando tre moduli tattici alternativi:
* 4-3-3
* 4-2-3-1
* 3-5-2

sulla base di vincoli funzionali e profili tattici specifici.

In [7]:
import pandas as pd

# Carica il file
df = pd.read_csv("Giocatori_Predetti_Con_Ruolo_Tattico.csv")

# Funzione di punteggio
def punteggio_totale(row):
    bonus = 100 if row["Etichetta_Predetta"] == "consigliato" else 50 if row["Etichetta_Predetta"] == "sorpresa" else 0
    return bonus + row["Score_norm"] * 10 + row["Prob_Gol"] * 5 + row["Pred_Gol_Pesata"] * 5

df["Punteggio_Totale"] = df.apply(punteggio_totale, axis=1)

squadre = df["Team"].unique()
risultati = []

for team in squadre:
    squadra_df = df[df["Team"] == team].copy()
    if squadra_df.empty:
        continue
    squadra_df["Punteggio_Totale"] = squadra_df.apply(punteggio_totale, axis=1)

    # Portiere
    portiere = squadra_df[squadra_df["Role"] == "P"].sort_values("Punteggio_Totale", ascending=False).head(1)

    # 4-3-3
    difensori = squadra_df[squadra_df["Role"] == "D"]
    terzini = difensori[difensori["Etichetta_Tattica"].isin(["Terzino offensivo", "Libero", "N.C."])].sort_values("Punteggio_Totale", ascending=False).head(2)
    centrali = difensori[difensori["Etichetta_Tattica"].isin(["Regista difensivo", "Leader difensivo", "Stopper", "N.C.", "Libero", "Terzino offensivo"]) & ~difensori["Player"].isin(terzini["Player"])].sort_values("Punteggio_Totale", ascending=False).head(2)
    dif_433 = pd.concat([terzini, centrali])

    centro = squadra_df[squadra_df["Role"] == "C"].sort_values("Punteggio_Totale", ascending=False)
    centro1 = centro.iloc[0]
    centro23 = centro[centro["Etichetta_Tattica"] != centro1["Etichetta_Tattica"]].head(2) if len(centro) >= 3 else centro.iloc[1:3]
    centro_433 = pd.concat([pd.DataFrame([centro1]), centro23])

    att = squadra_df[squadra_df["Role"] == "A"]

    # 1. ATTACCANTE CENTRALE
    att_centrale = att[att["Etichetta_Tattica"].isin(["Finalizzatore", "Attaccante di manovra", "Prima punta"])].sort_values("Punteggio_Totale", ascending=False).head(1)

    # 2. ATTACCANTI LARGHI
    # Escludi l’attaccante centrale selezionato
    att_rimanenti = att[~att["Player"].isin(att_centrale["Player"])]

    # Prova con larghi "ideali"
    att_larghi = att_rimanenti[att_rimanenti["Etichetta_Tattica"].isin(["Attaccante completo / Ala", "Seconda punta"])].sort_values("Punteggio_Totale", ascending=False).head(2)

    # Se mancano, integra con centrali
    if len(att_larghi) < 2:
        mancanti = 2 - len(att_larghi)
        larghi_secondari = att_rimanenti[
            att_rimanenti["Etichetta_Tattica"].isin(["Finalizzatore", "Attaccante di manovra", "Prima punta"]) &
            ~att_rimanenti["Player"].isin(att_larghi["Player"])
        ].sort_values("Punteggio_Totale", ascending=False).head(mancanti)
        att_larghi = pd.concat([att_larghi, larghi_secondari])

    # Se ancora mancano, usa N.C.
    if len(att_larghi) < 2:
        mancanti = 2 - len(att_larghi)
        larghi_nc = att_rimanenti[
            (att_rimanenti["Etichetta_Tattica"] == "N.C.") &
            ~att_rimanenti["Player"].isin(att_larghi["Player"])
        ].sort_values("Punteggio_Totale", ascending=False).head(mancanti)
        att_larghi = pd.concat([att_larghi, larghi_nc])

    # 3. Combina attacco completo
    att_433 = pd.concat([att_larghi, att_centrale])
    formazione_433 = pd.concat([portiere, dif_433, centro_433, att_433])
    score_433 = formazione_433["Punteggio_Totale"].sum()


    # 4-2-3-1
    centrocentrali = squadra_df[
        (squadra_df["Role"] == "C") &
        (squadra_df["Etichetta_Tattica"].isin(["Regista", "Centrale equilibrato", "Box-to-Box"]))
    ].sort_values("Punteggio_Totale", ascending=False).head(2)

    # Se meno di 2, fallback su Mezzala offensiva
    if len(centrocentrali) < 2:
        mancanti = 2 - len(centrocentrali)
        mezzali = squadra_df[
            (squadra_df["Role"] == "C") &
            (squadra_df["Etichetta_Tattica"] == "Mezzala offensiva") &
            (~squadra_df["Player"].isin(centrocentrali["Player"]))
        ].sort_values("Punteggio_Totale", ascending=False).head(mancanti)
        centrocentrali = pd.concat([centrocentrali, mezzali])

    # Se ancora meno di 2, fallback su N.C.
    if len(centrocentrali) < 2:
        mancanti = 2 - len(centrocentrali)
        nc = squadra_df[
            (squadra_df["Role"] == "C") &
            (squadra_df["Etichetta_Tattica"] == "N.C.") &
            (~squadra_df["Player"].isin(centrocentrali["Player"]))
        ].sort_values("Punteggio_Totale", ascending=False).head(mancanti)
        centrocentrali = pd.concat([centrocentrali, nc])


    tq1 = squadra_df[(squadra_df["Role"].isin(["C", "A"])) &
      (squadra_df["Etichetta_Tattica"].isin(["Trequartista", "Mezzala offensiva", "Seconda punta"]))].sort_values("Punteggio_Totale", ascending=False).head(1)

      # Escludi già selezionato tq1
    tq_esclusi = tq1["Player"]

    # 1. PRIORITÀ: Trequartista, Attaccante completo / Ala, Seconda punta
    tq2 = squadra_df[
        (squadra_df["Role"].isin(["C", "A"])) &
        (squadra_df["Etichetta_Tattica"].isin(["Trequartista", "Attaccante completo / Ala", "Seconda punta"])) &
        (~squadra_df["Player"].isin(tq_esclusi))
    ].sort_values("Punteggio_Totale", ascending=False).head(2)

    # 2. Se mancanti, integra con Mezzala offensiva
    if len(tq2) < 2:
        mancanti = 2 - len(tq2)
        supplementari_mezzali = squadra_df[
            (squadra_df["Role"].isin(["C", "A"])) &
            (squadra_df["Etichetta_Tattica"] == "Mezzala offensiva") &
            (~squadra_df["Player"].isin(pd.concat([tq1, tq2])["Player"]))
        ].sort_values("Punteggio_Totale", ascending=False).head(mancanti)
        tq2 = pd.concat([tq2, supplementari_mezzali])

    # 3. Se ancora mancanti, integra con N.C.
    if len(tq2) < 2:
        mancanti = 2 - len(tq2)
        supplementari_nc = squadra_df[
            (squadra_df["Role"].isin(["C", "A"])) &
            (squadra_df["Etichetta_Tattica"] == "N.C.") &
            (~squadra_df["Player"].isin(pd.concat([tq1, tq2])["Player"]))
        ].sort_values("Punteggio_Totale", ascending=False).head(mancanti)
        tq2 = pd.concat([tq2, supplementari_nc])

    trequartisti = pd.concat([tq1, tq2])

    punta = squadra_df[(squadra_df["Role"] == "A") & (squadra_df["Etichetta_Tattica"].isin(["Finalizzatore", "Attaccante di manovra", "Prima punta"])) & (~squadra_df["Player"].isin(trequartisti["Player"]))].sort_values("Punteggio_Totale", ascending=False).head(1)

    formazione_4231 = pd.concat([portiere, dif_433, centrocentrali, trequartisti, punta])
    score_4231 = formazione_4231["Punteggio_Totale"].sum()

    # 3-5-2
    # === DIFENSORI 3-5-2 (prima centrali migliori, poi terzini/liberi agli esterni) ===
    dif_352 = difensori[difensori["Etichetta_Tattica"].isin([
        "Leader difensivo", "Regista difensivo", "Stopper", "Libero", "N.C.", "Terzino offensivo"
    ])].sort_values("Punteggio_Totale", ascending=False).copy()

    # Seleziona i 3 migliori centrali
    centrali_352_df = dif_352.head(3).copy()
    centrali_352_players = centrali_352_df["Player"].tolist()

    # Candidati per esterni: i migliori terzini o liberi non usati
    spostabili_esterni_df = dif_352[
        (dif_352["Etichetta_Tattica"].isin(["Terzino offensivo", "Libero"])) &
        (~dif_352["Player"].isin(centrali_352_players))
    ].sort_values("Punteggio_Totale", ascending=False).drop_duplicates(subset="Player").head(2)

    difensori_gia_usati = centrali_352_players
    giocatori_per_esterni = spostabili_esterni_df.to_dict("records")

    # === CENTROCAMPO: playmaker e interni ===
    playmaker = squadra_df[
        (squadra_df["Role"] == "C") &
        (squadra_df["Etichetta_Tattica"].isin(["Centrale equilibrato", "Regista", "Box-to-Box"]))
    ].sort_values("Punteggio_Totale", ascending=False).head(1)

    interni = squadra_df[
        (squadra_df["Role"] == "C") &
        (squadra_df["Etichetta_Tattica"].isin(["Mezzala offensiva", "Box-to-Box", "Centrale equilibrato"])) &
        (~squadra_df["Player"].isin(playmaker["Player"]))
    ].sort_values("Punteggio_Totale", ascending=False).head(2)

    for fallback in ["Regista", "Trequartista", "N.C."]:
        if len(interni) < 2:
            mancanti = 2 - len(interni)
            aggiunti = squadra_df[
                (squadra_df["Role"] == "C") &
                (squadra_df["Etichetta_Tattica"] == fallback) &
                (~squadra_df["Player"].isin(pd.concat([playmaker, interni])["Player"]))
            ].sort_values("Punteggio_Totale", ascending=False).head(mancanti)
            interni = pd.concat([interni, aggiunti])

    # === ESTERNI ===
    giocatori_per_esterni_filtrati = [p for p in giocatori_per_esterni if "Player" in p]
    esterni = pd.DataFrame(giocatori_per_esterni_filtrati)

    if "Player" not in esterni.columns or esterni.empty:
        giocatori_esterni_scelti = []
    else:
        giocatori_esterni_scelti = esterni["Player"].tolist()

    esterni_mancanti = 2 - len(esterni)

    # Se mancano ancora esterni, prova con altri ruoli compatibili
    if esterni_mancanti > 0:
        altri_esterni = squadra_df[
            (squadra_df["Role"].isin(["D", "C", "A"])) &
            (squadra_df["Etichetta_Tattica"].isin(["Terzino offensivo", "Attaccante completo / Ala", "Trequartista", "N.C."])) &
            (~squadra_df["Player"].isin(
                difensori_gia_usati + giocatori_esterni_scelti +
                playmaker["Player"].tolist() + interni["Player"].tolist()
            ))
        ].sort_values("Punteggio_Totale", ascending=False).drop_duplicates(subset="Player").head(esterni_mancanti)

        esterni = pd.concat([esterni, altri_esterni])

    esterni = esterni.drop_duplicates(subset="Player").head(2)

    # === RECUPERO ESTERNI MANCANTI CON TERZINI/LIBERI NON USATI ===
    esterni_mancanti = 2 - len(esterni)
    if esterni_mancanti > 0:
        gia_usati = (
            centrali_352_df["Player"].tolist() +
            interni["Player"].tolist() +
            playmaker["Player"].tolist() +
            esterni["Player"].tolist()
        )

        terzini_liberi_extra = squadra_df[
            (squadra_df["Etichetta_Tattica"].isin(["Terzino offensivo", "Libero"])) &
            (~squadra_df["Player"].isin(gia_usati))
        ].sort_values("Punteggio_Totale", ascending=False).drop_duplicates(subset="Player").head(esterni_mancanti)

        esterni = pd.concat([esterni, terzini_liberi_extra]).drop_duplicates(subset="Player").head(2)

    # === CENTROCAMPO COMPLETO
    centrocampo_352 = pd.concat([playmaker, interni, esterni])

    # === ATTACCANTI
    att_352 = squadra_df[
        (squadra_df["Role"] == "A") &
        (~squadra_df["Player"].isin(centrocampo_352["Player"]))
    ].sort_values("Punteggio_Totale", ascending=False).head(2)

    # === FORMAZIONE COMPLETA
    formazione_352 = pd.concat([portiere, centrali_352_df, centrocampo_352, att_352])
    score_352 = formazione_352["Punteggio_Totale"].sum()

     # Scegli il modulo migliore e stampa tutto
    print(f"\n========================= TEAM: {team} =========================")

    print(f"\n MODULO 4-3-3 – SCORE: {score_433:.2f}")
    display(formazione_433[["Player", "Role", "Etichetta_Tattica"]].reset_index(drop=True))

    print(f"\n MODULO 4-2-3-1 – SCORE: {score_4231:.2f}")
    display(formazione_4231[["Player", "Role", "Etichetta_Tattica"]].reset_index(drop=True))

    print(f"\n MODULO 3-5-2 – SCORE: {score_352:.2f}")
    display(formazione_352[["Player", "Role", "Etichetta_Tattica"]].reset_index(drop=True))

    if score_433 >= score_4231 and score_433 >= score_352:
        migliore = "4-3-3"
        formazione_migliore = formazione_433
        score_migliore = score_433
    elif score_4231 >= score_433 and score_4231 >= score_352:
        migliore = "4-2-3-1"
        formazione_migliore = formazione_4231
        score_migliore = score_4231
    else:
        migliore = "3-5-2"
        formazione_migliore = formazione_352
        score_migliore = score_352

    print(f"\n MIGLIOR MODULO PER {team}: {migliore} – SCORE: {score_migliore:.2f}")
    display(formazione_migliore[["Player", "Role", "Etichetta_Tattica"]].reset_index(drop=True))

    formazione_migliore = formazione_migliore.copy()
    formazione_migliore["Team"] = team
    formazione_migliore["Modulo"] ="'" + migliore
    formazione_migliore["Score_Formazione"] = score_migliore
    risultati.append(formazione_migliore)

# Visualizza e scarica il CSV
from google.colab import files
# Unione risultati finali
df_finale = pd.concat(risultati).reset_index(drop=True)
df_finale["Modulo"] = df_finale["Modulo"].astype(str)

# Seleziona solo Player, Team, Role, Modulo
df_finale_ridotto = df_finale[["Player", "Team", "Role", "Modulo"]]

# Visualizza e scarica il CSV
df_finale_ridotto.to_csv("Formazioni_Ottimali_Per_Squadra.csv", index=False)
files.download("Formazioni_Ottimali_Per_Squadra.csv")



========================= TEAM: Torino =========================

 MODULO 4-3-3 – SCORE: 732.50


,Player,Role,Etichetta_Tattica
0,vanja milinkovic-savic,P,Portiere
1,raoul bellanova,D,Terzino offensivo
2,ricardo rodriguez,D,Libero
3,alessandro buongiorno,D,Leader difensivo
4,valentino lazaro,D,Libero
5,ivan ilic,C,Trequartista
6,samuele ricci,C,Regista
7,adrien tameze,C,Centrale equilibrato
8,nikola vlasic,A,Seconda punta
9,pietro pellegri,A,Attaccante di manovra



 MODULO 4-2-3-1 – SCORE: 680.32


,Player,Role,Etichetta_Tattica
0,vanja milinkovic-savic,P,Portiere
1,raoul bellanova,D,Terzino offensivo
2,ricardo rodriguez,D,Libero
3,alessandro buongiorno,D,Leader difensivo
4,valentino lazaro,D,Libero
5,samuele ricci,C,Regista
6,adrien tameze,C,Centrale equilibrato
7,ivan ilic,C,Trequartista
8,nikola vlasic,A,Seconda punta
9,zanos savva,A,N.C.



 MODULO 3-5-2 – SCORE: 630.04


...

,Player,Role,Etichetta_Tattica
0,guillermo ochoa,P,Portiere
1,domagoj bradaric,D,Libero
2,lorenzo pirola,D,Leader difensivo
3,norbert gyomber,D,Regista difensivo
4,giulio maggiore,C,Regista
5,loum tchaouna,C,Mezzala offensiva
6,grigoris kastanos,C,Mezzala offensiva
7,alessandro zanoli,D,Libero
8,antonio candreva,C,Trequartista
9,simy,A,N.C.



 MIGLIOR MODULO PER Salernitana: 4-3-3 – SCORE: 402.62


,Player,Role,Etichetta_Tattica
0,guillermo ochoa,P,Portiere
1,domagoj bradaric,D,Libero
2,alessandro zanoli,D,Libero
3,lorenzo pirola,D,Leader difensivo
4,norbert gyomber,D,Regista difensivo
5,antonio candreva,C,Trequartista
6,giulio maggiore,C,Regista
7,junior sambia,C,N.C.
8,simy,A,N.C.
9,boulaye dia,A,N.C.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Generiamo la top 11 per la 39esima giornata di tutta la serie A, con i 3 moduli diversi e scegliendo poi la migliore

In [8]:
import pandas as pd
from google.colab import files

# Carica il dataset
df = pd.read_csv("Giocatori_Predetti_Con_Ruolo_Tattico.csv")

# Calcolo punteggio totale
def punteggio_totale(row):
    bonus = 100 if row["Etichetta_Predetta"] == "consigliato" else 50 if row["Etichetta_Predetta"] == "sorpresa" else 0
    return bonus + row["Score_norm"] * 10 + row["Prob_Gol"] * 5 + row["Pred_Gol_Pesata"] * 5

df["Punteggio_Totale"] = df.apply(punteggio_totale, axis=1)

# Portiere
portiere = df[df["Role"] == "P"].sort_values("Punteggio_Totale", ascending=False).head(1)

# --- MODULO 4-3-3 ---
difensori = df[df["Role"] == "D"]
terzini = difensori[difensori["Etichetta_Tattica"].isin(["Terzino offensivo", "Libero", "N.C."])].sort_values("Punteggio_Totale", ascending=False).head(2)
centrali = difensori[difensori["Etichetta_Tattica"].isin(["Regista difensivo", "Leader difensivo", "Stopper", "Libero", "N.C.", "Terzino offensivo"]) &
                     ~difensori["Player"].isin(terzini["Player"])].sort_values("Punteggio_Totale", ascending=False).head(2)
dif_433 = pd.concat([terzini, centrali])

centro = df[df["Role"] == "C"].sort_values("Punteggio_Totale", ascending=False)
centro1 = centro.iloc[0]
centro23 = centro[centro["Etichetta_Tattica"] != centro1["Etichetta_Tattica"]].head(2) if len(centro) >= 3 else centro.iloc[1:3]
centro_433 = pd.concat([pd.DataFrame([centro1]), centro23])

att = df[df["Role"] == "A"]
att_centrale = att[att["Etichetta_Tattica"].isin(["Finalizzatore", "Attaccante di manovra", "Prima punta"])].sort_values("Punteggio_Totale", ascending=False).head(1)
att_rimanenti = att[~att["Player"].isin(att_centrale["Player"])]
att_larghi = att_rimanenti[att_rimanenti["Etichetta_Tattica"].isin(["Attaccante completo / Ala", "Seconda punta"])].sort_values("Punteggio_Totale", ascending=False).head(2)

if len(att_larghi) < 2:
    mancanti = 2 - len(att_larghi)
    larghi_extra = att_rimanenti[att_rimanenti["Etichetta_Tattica"].isin(["Finalizzatore", "Attaccante di manovra", "Prima punta"]) &
                                 ~att_rimanenti["Player"].isin(att_larghi["Player"])].sort_values("Punteggio_Totale", ascending=False).head(mancanti)
    att_larghi = pd.concat([att_larghi, larghi_extra])

att_433 = pd.concat([att_larghi, att_centrale])
formazione_433 = pd.concat([portiere, dif_433, centro_433, att_433])
formazione_433["Modulo"] = "'4-3-3"
score_433 = formazione_433["Punteggio_Totale"].sum()

# --- MODULO 4-2-3-1 ---
centrocentrali = df[(df["Role"] == "C") & (df["Etichetta_Tattica"].isin(["Regista", "Centrale equilibrato", "Box-to-Box"]))].sort_values("Punteggio_Totale", ascending=False).head(2)
if len(centrocentrali) < 2:
    mancanti = 2 - len(centrocentrali)
    mezzali = df[(df["Role"] == "C") & (df["Etichetta_Tattica"] == "Mezzala offensiva") & (~df["Player"].isin(centrocentrali["Player"]))].sort_values("Punteggio_Totale", ascending=False).head(mancanti)
    centrocentrali = pd.concat([centrocentrali, mezzali])

tq1 = df[(df["Role"].isin(["C", "A"])) & (df["Etichetta_Tattica"].isin(["Trequartista", "Mezzala offensiva", "Seconda punta"]))].sort_values("Punteggio_Totale", ascending=False).head(1)
tq2 = df[(df["Role"].isin(["C", "A"])) & (df["Etichetta_Tattica"].isin(["Trequartista", "Attaccante completo / Ala", "Seconda punta"])) &
         (~df["Player"].isin(tq1["Player"]))].sort_values("Punteggio_Totale", ascending=False).head(2)
trequartisti = pd.concat([tq1, tq2])

punta = df[(df["Role"] == "A") & (df["Etichetta_Tattica"].isin(["Finalizzatore", "Attaccante di manovra", "Prima punta"])) &
           (~df["Player"].isin(trequartisti["Player"]))].sort_values("Punteggio_Totale", ascending=False).head(1)

formazione_4231 = pd.concat([portiere, dif_433, centrocentrali, trequartisti, punta])
formazione_4231["Modulo"] = "'4-2-3-1"
score_4231 = formazione_4231["Punteggio_Totale"].sum()

# --- MODULO 3-5-2 ---
dif_352 = difensori[difensori["Etichetta_Tattica"].isin(["Leader difensivo", "Regista difensivo", "Stopper", "Libero", "N.C.", "Terzino offensivo"])].sort_values("Punteggio_Totale", ascending=False)
centrali_352 = dif_352.head(3)
spostabili_esterni = dif_352[dif_352["Etichetta_Tattica"].isin(["Terzino offensivo", "Libero"]) &
                             (~dif_352["Player"].isin(centrali_352["Player"]))].sort_values("Punteggio_Totale", ascending=False).head(2)

playmaker = df[(df["Role"] == "C") & (df["Etichetta_Tattica"].isin(["Regista", "Centrale equilibrato", "Box-to-Box"]))].sort_values("Punteggio_Totale", ascending=False).head(1)
interni = df[(df["Role"] == "C") & (df["Etichetta_Tattica"].isin(["Mezzala offensiva", "Box-to-Box", "Centrale equilibrato"])) &
             (~df["Player"].isin(playmaker["Player"]))].sort_values("Punteggio_Totale", ascending=False).head(2)

esterni = spostabili_esterni.drop_duplicates(subset="Player").head(2)
if len(esterni) < 2:
    mancanti = 2 - len(esterni)
    extra = df[df["Etichetta_Tattica"].isin(["Terzino offensivo", "Libero", "N.C."]) &
               (~df["Player"].isin(esterni["Player"]))].sort_values("Punteggio_Totale", ascending=False).head(mancanti)
    esterni = pd.concat([esterni, extra]).drop_duplicates(subset="Player").head(2)

centrocampo_352 = pd.concat([playmaker, interni, esterni])
att_352 = df[(df["Role"] == "A") & (~df["Player"].isin(centrocampo_352["Player"]))].sort_values("Punteggio_Totale", ascending=False).head(2)

formazione_352 = pd.concat([portiere, centrali_352, centrocampo_352, att_352])
formazione_352["Modulo"] = "'3-5-2"
score_352 = formazione_352["Punteggio_Totale"].sum()

# --- CONFRONTO E OUTPUT ---
print(f"\n MODULO 4-3-3 – SCORE: {score_433:.2f}")
display(formazione_433[["Player", "Team", "Role", "Etichetta_Tattica"]])

print(f"\n MODULO 4-2-3-1 – SCORE: {score_4231:.2f}")
display(formazione_4231[["Player", "Team", "Role", "Etichetta_Tattica"]])

print(f"\n MODULO 3-5-2 – SCORE: {score_352:.2f}")
display(formazione_352[["Player", "Team", "Role", "Etichetta_Tattica"]])

if score_433 >= score_4231 and score_433 >= score_352:
    migliore = "4-3-3"
    score_migliore = score_433
elif score_4231 >= score_433 and score_4231 >= score_352:
    migliore = "4-2-3-1"
    score_migliore = score_4231
else:
    migliore = "3-5-2"
    score_migliore = score_352

print(f"\n MIGLIOR MODULO DEL CAMPIONATO: {migliore} – SCORE: {score_migliore:.2f}")

# Unione e salvataggio finale
df_finale = pd.concat([formazione_433, formazione_4231, formazione_352]).reset_index(drop=True)

# Esporta solo colonne desiderate con modulo protetto da interpretazione Excel
df_finale_ridotto = df_finale[["Player", "Team", "Role", "Modulo"]]
df_finale_ridotto.to_csv("Formazioni_Top_SerieA_Tre_Moduli.csv", index=False)
files.download("Formazioni_Top_SerieA_Tre_Moduli.csv")




 MODULO 4-3-3 – SCORE: 1223.38


,Player,Team,Role,Etichetta_Tattica
0,vanja milinkovic-savic,Torino,P,Portiere
77,riccardo calafiori,Bologna,D,Terzino offensivo
76,alessandro bastoni,Inter,D,Terzino offensivo
78,francesco acerbi,Inter,D,Regista difensivo
80,amir rrahmani,Napoli,D,Terzino offensivo
302,hakan calhanoglu,Inter,C,Trequartista
308,arthur,Fiorentina,C,Centrale equilibrato
306,matteo pessina,Monza,C,Centrale equilibrato
517,albert gudmundsson,Genoa,A,Attaccante completo / Ala
523,rafael leao,AC Milan,A,Attaccante completo / Ala



 MODULO 4-2-3-1 – SCORE: 1223.38


,Player,Team,Role,Etichetta_Tattica
0,vanja milinkovic-savic,Torino,P,Portiere
77,riccardo calafiori,Bologna,D,Terzino offensivo
76,alessandro bastoni,Inter,D,Terzino offensivo
78,francesco acerbi,Inter,D,Regista difensivo
80,amir rrahmani,Napoli,D,Terzino offensivo
308,arthur,Fiorentina,C,Centrale equilibrato
306,matteo pessina,Monza,C,Centrale equilibrato
302,hakan calhanoglu,Inter,C,Trequartista
517,albert gudmundsson,Genoa,A,Attaccante completo / Ala
523,rafael leao,AC Milan,A,Attaccante completo / Ala



 MODULO 3-5-2 – SCORE: 1217.93


,Player,Team,Role,Etichetta_Tattica
0,vanja milinkovic-savic,Torino,P,Portiere
77,riccardo calafiori,Bologna,D,Terzino offensivo
78,francesco acerbi,Inter,D,Regista difensivo
76,alessandro bastoni,Inter,D,Terzino offensivo
308,arthur,Fiorentina,C,Centrale equilibrato
306,matteo pessina,Monza,C,Centrale equilibrato
330,mattia zaccagni,Lazio,C,Box-to-Box
80,amir rrahmani,Napoli,D,Terzino offensivo
86,federico dimarco,Inter,D,Terzino offensivo
516,lautaro martinez,Inter,A,Finalizzatore



 MIGLIOR MODULO DEL CAMPIONATO: 4-3-3 – SCORE: 1223.38


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>